## The System:

![grafik.png][def]


[def]: .\PTG_System.png

## Preperation
#### Welche Packages brauchen wir NICHT?

In [2]:
!pip install gurobipy==10.0.0
!pip install pyomo==6.6.2
!pip install numpy==1.24.3
!pip install pandas==1.3.5
#!pip install tsam==2.3.1
!pip install matplotlib
#!pip install july

In [3]:
from gurobipy import *
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import tsam.timeseriesaggregation as tsam
#import july
#from july.utils import date_range


## Time series
#### Daten laden

In [4]:
data_sheet = pd.read_excel(".\Data_Sheet_neu.xlsx")
print(data_sheet)

time_slices = np.array(data_sheet.index)
#print(time_slices)


time_series = list(range(1, 8761))
next_time_slice = {time_series[i]: time_series[(i + 1) % len(time_series)] for i in range(len(time_series))}
#print(next_time_slice)


# gas price of each time slice
power_demand_time_series = dict(zip(time_slices, np.array(data_sheet['PowerD_demand'])))

# heat demand of each time slice
heat_demand_time_series = dict(zip(time_slices, np.array(data_sheet['HeatD_demand'])))

# electricity_price of each time slice
electricity_price_time_series = dict(zip(time_slices, np.array(data_sheet['electricity_price'])))

# PV_Power of eacj time slice
power_PV_time_series = dict(zip(time_slices, np.array(data_sheet['P_PV_tot'])))



      Unnamed: 0  s     t  dt  PowerD_demand  HeatD_demand  P_PV_tot  \
0              0  1     1   1          158.4     69.178754   0.00000   
1              1  1     2   1          156.0     56.480866   0.00000   
2              2  1     3   1          156.0     45.267146   0.00000   
3              3  1     4   1          156.6     39.165564   0.00000   
4              4  1     5   1          151.2     40.567279   0.00000   
...          ... ..   ...  ..            ...           ...       ...   
8755        8755  1  8756   1          168.6     15.307163   4.33854   
8756        8756  1  8757   1          174.0     14.920863   0.00000   
8757        8757  1  8758   1          167.4     14.293124   0.00000   
8758        8758  1  8759   1          166.8     23.033177   0.00000   
8759        8759  1  8760   1          160.2     32.497543   0.00000   

      electricity_price  
0             -0.013287  
1             -0.002750  
2             -0.003778  
3             -0.013056  
4    

## Setting up the model:

In [5]:
model = ConcreteModel()

# declare sets
model.time_slices = set(time_slices)

In [6]:
# declare decision variables
model.OPEX = Var(domain=NonNegativeReals) # [€]

# declare time dependent decision variables
model.P_e_buy = Var(model.time_slices, domain=NonNegativeReals) # Power, that needs to get bought from the grid [kW_e]
model.P_CH4_sell = Var(model.time_slices, domain=NonNegativeReals) # CH4, that gets to sell to the grid [kW_CH4]
model.mdot_CO2_atm = Var(model.time_slices, domain=NonNegativeReals) # CH4, that gets to sell to the grid [kW_CH4]

# Component_Inputs
model.P_e_PEM = Var(model.time_slices, domain=NonNegativeReals) # Electrolysis [kW_e]
model.P_e_DAC = Var(model.time_slices, domain=NonNegativeReals) # DirectAirCapture [kW_e]
model.P_H2_MR = Var(model.time_slices, domain=NonNegativeReals) # MethaneReformation [kW_H2]
model.P_CO2_MR = Var(model.time_slices, domain=NonNegativeReals) # MethaneReformation [kW_CO2] STIMMT EINHEIT???
model.P_CH4_CHP = Var(model.time_slices, domain=NonNegativeReals) # CombinedHeatPower [kW_CH4]
model.P_CH4_BO = Var(model.time_slices, domain=NonNegativeReals) # Boiler [kW_CH4]

model.y_PEM = Var(model.time_slices, domain=Binary) # Electrolysis, on/off
model.y_CHP = Var(model.time_slices, domain=Binary) # CombinedHeatPower, on/off
model.y_BO = Var(model.time_slices, domain=Binary) # Boiler, on/off

# Component_Outputs
model.P_e_PV = Var(model.time_slices, domain=NonNegativeReals) # PV [kW_e]
model.P_H2_PEM = Var(model.time_slices, domain=NonNegativeReals) # Electrolysis [kW_H2]
model.mdot_CO2_DAC = Var(model.time_slices, domain=NonNegativeReals) # DirectAirCapture [kg_CO2/hr]
model.P_CH4_MR = Var(model.time_slices, domain=NonNegativeReals) # MethaneReformation [kW_CH4]
model.P_e_CHP = Var(model.time_slices, domain=NonNegativeReals) # CombinedHeatPower [kW_e]
model.P_h_CHP = Var(model.time_slices, domain=NonNegativeReals) # CombinedHeatPower [kW_h]
model.P_h_BO = Var(model.time_slices, domain=NonNegativeReals) # Boiler [kW_h]

# Storages
model.P_e_battery = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kW_e] 
model.storage_level_battery = Var(model.time_slices, domain=NonNegativeReals) # [kWh]
model.in_battery =  Var(model.time_slices, domain=Binary) #einspeichern
model.out_battery = Var(model.time_slices, domain=Binary) #ausspeichern

model.P_h_thermalStorage = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kW_h]
model.storage_level_thermalStorage = Var(model.time_slices, domain=NonNegativeReals) # [kWh]
model.in_thermalStorage =  Var(model.time_slices, domain=Binary) #einspeichern
model.out_thermalStorage = Var(model.time_slices, domain=Binary) #ausspeichern

model.P_H2_gasStorage = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kW_H2]
model.storage_level_H2Storage = Var(model.time_slices, domain=NonNegativeReals) # [kWh_H2]
model.in_H2Storage =  Var(model.time_slices, domain=Binary) #einspeichern
model.out_H2Storage = Var(model.time_slices, domain=Binary) #ausspeichern

model.P_CO2_gasStorage = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kg_CO2/hr]
model.storage_level_CO2Storage = Var(model.time_slices, domain=NonNegativeReals) # [kg_CO2]
model.in_CO2Storage =  Var(model.time_slices, domain=Binary) #einspeichern
model.out_CO2Storage = Var(model.time_slices, domain=Binary) #ausspeichern

model.P_CH4_gasStorage = Var(model.time_slices, domain=Reals) # [negative: storage charge, positive: storage discharge] [kW_CH4]
model.storage_level_CH4Storage = Var(model.time_slices, domain=NonNegativeReals) # [kWh_CH4]
model.in_CH4Storage =  Var(model.time_slices, domain=Binary) #einspeichern
model.out_CH4Storage = Var(model.time_slices, domain=Binary) #ausspeichern


In [7]:
# declare parameters

model.price_co2 = 84.4 # [€/ton]
model.price_gas = 0.0775 # [€/kWh]


model.P_e_N_PV = 900 #[kW_e]

model.P_e_N_PEM = 900 #[kW_e]
model.eta_N_PEM = 0.63 #eta muss noch linerarisiert werden!!
model.p_min_PEM = 0.2
model.p_max_PEM = 1.2

model.mdot_CH4_N_DAC = 250 #[kg_CO2/hr]
model.eta_DAC = 0.5 #[kg_CO2/kWh_e]

model.P_CH4_N_MR = 1500 #[kW_CH4]
model.eta_MR = 0.78
model.co2need_MR = 0.178 #E_in_CO2/E_out_CH4

model.P_h_N_CHP = 800 #[kW_h]
model.eta__N_CHP_Q = 0.47 #eta muss noch linerarisiert werden!!
model.eta_N_CHP_P = 0.4 #eta muss noch linerarisiert werden!!
model.p_min_CHP = 0.5
model.p_max_CHP = 1

model.P_h_N_BO = 500 #[kW_h]
model.eta_N_BO = 0.9 #eta muss noch linerarisiert werden!!
model.p_min_BO = 0.001
model.p_max_BO = 1

model.E_N_battery = 5000 #[kWh_e]
model.eta_battery = 0.92 
model.loss_battery = 0.000042 #[1/hr]
model.c_battery = 0.36 #[1/hr]

model.Q_N_thermalStorage = 5000 #[kWh_Q]
model.eta_thermalStorage = 0.95 
model.loss_thermalStorage = 0.005 #[1/hr]
model.c_thermalStorage = 1 #[1/hr]

model.E_N_H2Storage = 5000 #[kWh_H2]
model.m_N_CO2Storage = 5000 #[kg_CO2]
model.E_N_CH4Storage = 5000 #[kWh_CH4]
model.loss_gasTanks = 0 #[1/hr]
model.eta_gasTanks = 1 
model.c_gasTanks = 0.25 #[1/hr]


In [8]:
#declare time dependent parameters
model.P_e_demand = Param(model.time_slices, initialize = power_demand_time_series)
model.Q_demand = Param(model.time_slices, initialize = heat_demand_time_series)

model.electricity_price = Param(model.time_slices, initialize = electricity_price_time_series)

model.P_PV = Param(model.time_slices, initialize = power_PV_time_series)


In [10]:
# declare unindexed constraints
model.OPEX_constraint = Constraint(expr =  model.OPEX == sum(model.electricity_price[time_slice]*model.P_e_buy[time_slice] for time_slice in model.time_slices) 
                                     +  sum(model.price_co2 * model.mdot_CO2_atm[time_slice] for time_slice in model.time_slices)
                                     -  sum(model.price_gas * model.P_CH4_sell[time_slice] for time_slice in model.time_slices)
                                     )

# declare rules to setup indexed constraints
def electricty_balance_constraint_rule(model, time_slice):
    return model.P_e_PV[time_slice] + model.P_e_CHP[time_slice] + model.P_e_battery[time_slice] + model.P_e_buy[time_slice] == model.P_e_demand[time_slice] + model.P_e_PEM[time_slice] + model.P_e_DAC[time_slice]

def heat_balance_constraint_rule(model, time_slice):
    return model.P_h_CHP[time_slice] + model.P_h_BO[time_slice] + model.P_h_thermalStorage[time_slice] == model.Q_demand

def h2_balance_constraint_rule(model, time_slice):
    return model.P_H2_PEM[time_slice] + model.P_H2_gasStorage == model.P_H2_MR

def co2_balance_contraint_rule(model, time_slices):
    return model.mdot_CO2_DAC[time_slices] + model.P_CO2_gasStorage[time_slices] == model.P_CO2_MR[time_slices]
# gesmischte EINHEITEN !!!ACHTUNG

def ch4_balance_constraint_rule(model, time_slices):
    return model.P_CH4_MR[time_series] + model.P_CH4_gasStorage[time_series] == model.P_CH4_MR[time_series] + model.P_CH4_BO[time_series] + model.P_CH4_sell[time_series]

##Components, Input->Output, Nenngröße, 


##Storages, storage_level, storage_maximum



# declare indexed constraints

## Solving the problem

In [ ]:
#SolverFactory('gurobi').solve(model).write()